In [1]:
import numpy as np
import pybamm
import matplotlib.pyplot as plt
from utils.generate_new_volume import generate_new_volume
from utils.physical_properties import *
from utils.plotting import plot_volume

# 4 phase volumes
phases = ['Pore','Crack','Particle','CBD']

# Volume generation
def generate_new_volume_rve(reduction, subregion_id, subregion):
    model_name = 'slicegan_r{}_s{}{}'.format(str(reduction).replace('.', 'p'), str(subregion_id), str(subregion))
    model_path = 'model/{}/{}'.format(model_name, model_name)

    return generate_new_volume(model_path=model_path)

# Diffusivity function to modify for Pybamm
def lico2_diffusivity(sto, T, coeff):
    """
    LiCo2 diffusivity as a function of stochiometry, in this case the
    diffusivity is taken to be a constant. The value is taken from Dualfoil [1].

    References
    ----------
    .. [1] http://www.cchem.berkeley.edu/jsngrp/fortran.html

    Parameters
    ----------
    sto: :class:`pybamm.Symbol`
        Electrode stochiometry
    T: :class:`pybamm.Symbol`
        Dimensional temperature

    Returns
    -------
    :class:`pybamm.Symbol`
        Solid diffusivity
    """
    #D_ref = 1 * 10 ** (-13)
    D_ref = (coeff / 0.4) * 10 ** (-13) # 0.4 is a random number picked from some GAN runs
    E_D_s = 18550
    arrhenius = pybamm.exp(E_D_s / pybamm.pybamm.constants.R * (1 / 298.15 - 1 / T))

    return D_ref * arrhenius

def properties_and_echem_from_volume(reduction, subregion_id, subregion, n=10, is_plot=True):
    capacities = np.zeros((n,1))
    vfs_particle = np.zeros((n,1))
    vfs_pore = np.zeros((n,1))
    D_effs = np.zeros((n,1))

    for i in range(n):
        vol = generate_new_volume_rve(reduction, subregion_id, subregion)
        '''
        Properties to modify:
        - Positive electrode porosity
        - Positive electrode active material volume fraction
        Things to work on later:
        - inputting Deff into positive electrode diffusivity (need to modify function in parameters)
        - inputting particle size (current volume is too small to have 'particles')
            - use surface area to reverse calculate particle size? particle size controls surface area in pybamm (sphere = 3/(r*(1-vf_pore)))
        '''
        # Set all cracks and pores equal for porosity
        vol[vol==1] = 0
        vf_particle = volume_fraction(vol, phase_class=2)
        vf_pore     = volume_fraction(vol, phase_class=0)
        # # pybamm can fail at extreme values
        # if vf_particle > 0.85:
        #     vf_particle = 0.85
        # if vf_pore < 0.09:
        #     vf_pore = 0.09

        # Diffusivity and tortuosity
        # Set all phases to 0 except particle for taufactor
        vol[vol==3] = 0
        vol[vol==2] = 1
        homogenized_properties = compute_tau_and_D_eff(vol, verbose=False)
        D_eff = homogenized_properties['D_eff'].item()

        vfs_particle[i] = vf_particle
        vfs_pore[i] = vf_pore
        D_effs[i] = D_eff

        # PyBaMM model
        model = pybamm.lithium_ion.DFN()
        # Default parameter set
        params = pybamm.ParameterValues("Marquis2019")
        # Modify parameters
        params['Positive electrode porosity'] = vf_pore
        params['Positive electrode active material volume fraction'] = vf_particle
        params['Positive electrode diffusivity [m2.s-1]'] = lambda sto, T: lico2_diffusivity(sto, T, D_eff)
        # Run the simulation
        sim = pybamm.Simulation(model,parameter_values=params)
        sim.solve([0, 7200],initial_soc=1)
        sol = sim.solution
        # Save the discharge capacity
        capacities[i] = sol['Discharge capacity [A.h]'].entries[-1]

        # Plot to see if the simulations are any different
        if is_plot:
            plt.plot(sol['Discharge capacity [A.h]'].entries, sol['Voltage [V]'].entries)

    if is_plot:
        plt.show()
    
    return capacities, vfs_particle, vfs_pore, D_effs


In [ ]:
reduction = 1.0
subregion = 1
subregion_id = 1

n = 10
